In [146]:
%pwd

'd:\\'

In [147]:
%pwd

'd:\\'

In [148]:
import os 
os.chdir("../")

In [149]:
from house_project.Config.config_entity import DataTransfomationConfig
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np 
from house_project import logging
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder,PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from house_project.Utility.common import *
from house_project.Utility.feature_engineering import FeatureEngineering
import os 
from pathlib import Path
import joblib

In [150]:
class DataTransfomation:
    def __init__(self):
        self.data_transform = DataTransfomationConfig()
        self.schema = Read_Yaml("End_to_End_House_Price_Estimater_IkmanData/schema.yaml")
        self.data_clean = FeatureEngineering()
        
        Create_Dir(self.data_transform.root_dir)
            
    def preprocess_step(self):
        
        num_columns = self.schema.get('scaler',[])
        cat_columns = self.schema.get('one_hot',[])
        power_col = self.schema.get('power',[])
        
        num_pipleine = Pipeline([
            ('scaler',StandardScaler())
        ])    
        
        cat_pipeline = Pipeline([
            ('one_hot',OneHotEncoder(handle_unknown='ignore'))
        ])
        
        power_pipleine=Pipeline([
            ('power',PowerTransformer(method='yeo-johnson'))
        ])
        
        preprocess = ColumnTransformer([
            ('power_pipleine',power_pipleine,power_col),
            ('num_pipeline',num_pipleine,num_columns),
            ('cat_pipeline',cat_pipeline,cat_columns)
            
        ])
        return preprocess
    def apply_feature_engineering_and_data_transfomation(self):
        try:
            #csv_file = check_csv_occur('End_to_End_House_Price_Estimater_IkmanData/artifacts/data_ingestion')
            
            data = pd.read_csv(Path("End_to_End_House_Price_Estimater_IkmanData/artifacts/data_ingestion/house_prices.csv"))
            
            drop_columns = self.schema.get("drop_columns",[])
            
            data = data.drop(columns=drop_columns,axis=1)
            
            
            data = self.data_clean.clean_bath(data)
            data = self.data_clean.clean_bed(data)
            data = self.data_clean.clean_land_size(data)
            data = self.data_clean.clean_house_size(data)
            data = self.data_clean.clean_price(data)
            data = self.data_clean.Geo_Address(data)
            logging.info("data cleaning Completed ")
            
            outlier_columns = self.schema.get('outlier',[])
            data = self.data_clean.remove_outliers_iqr(data=data,columns=outlier_columns)
            logging.info('Outlier removel Completed')
            logging.info(f"Shape after outlier: {data.shape}")
            preprocess_obj = self.preprocess_step()
            train_data,test_data = train_test_split(data,test_size=0.2,random_state=42)
            target_col = self.schema.get('TARGET',[])
            print(train_data.shape)
            print(test_data.shape)
            train_data_input_feature = train_data.drop(columns = target_col,axis = 1)
            train_data_target_feature = train_data[target_col]
            
            test_data_input_feature = test_data.drop(columns = target_col,axis = 1)
            test_data_target_feature = test_data[target_col]
            
            train_pre = preprocess_obj.fit_transform(train_data_input_feature)
            test_pre = preprocess_obj.transform(test_data_input_feature)
            print(train_pre.shape)
            print(test_pre.shape)
            
            #train_data_target_feature = np.array(train_data_target_feature).reshape(-1, 1)
            #test_data_target_feature = np.array(test_data_target_feature).reshape(-1, 1)

            print(f"train_pre shape: {train_pre.shape}")
            print(f"train_data_target_feature shape: {train_data_target_feature.shape}")

            train_arr = np.hstack([train_pre, train_data_target_feature])
            test_arr = np.hstack([test_pre, test_data_target_feature])

            
            
            return data
        except Exception as e:
            logging.error(f"Error in data cleaning {e}")
            raise e    

In [151]:
data_transfomation = DataTransfomation()
data_transfomation.apply_feature_engineering_and_data_transfomation()

[ 2025-02-18 17:11:57,120 ] 20 root - INFO - Read the End_to_End_House_Price_Estimater_IkmanData/schema.yaml Yaml file
[ 2025-02-18 17:11:57,123 ] 11 root - INFO - artifacts/data_transfomation Created Succesfully
[ 2025-02-18 17:11:57,417 ] 39 root - INFO - Shape before dropna: (15327, 6)
[ 2025-02-18 17:11:57,417 ] 42 root - INFO - Shape after dropna: (15291, 6)
[ 2025-02-18 17:11:57,478 ] 51 root - INFO - data cleaning Completed 
[ 2025-02-18 17:11:57,478 ] 71 root - INFO - Remove the outliers from ['Baths', 'Land size', 'House size', 'Price', 'Beds']
[ 2025-02-18 17:11:57,478 ] 71 root - INFO - Remove the outliers from ['Baths', 'Land size', 'House size', 'Price', 'Beds']
[ 2025-02-18 17:11:57,478 ] 71 root - INFO - Remove the outliers from ['Baths', 'Land size', 'House size', 'Price', 'Beds']
[ 2025-02-18 17:11:57,478 ] 71 root - INFO - Remove the outliers from ['Baths', 'Land size', 'House size', 'Price', 'Beds']
[ 2025-02-18 17:11:57,495 ] 71 root - INFO - Remove the outliers fro

(9588, 6)
(2397, 6)
(9588, 121)
(2397, 121)
train_pre shape: (9588, 121)
train_data_target_feature shape: (9588, 1)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)